<a href="https://colab.research.google.com/github/nirmit10000/YOLO_player_detection_dist_calc/blob/main/Tennis_Player_Tracking_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installations & Importing dependencies

In [1]:
!pip install opencv-python
!pip install ultralytics

#necessary dependencies installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [2]:
# cv2 for annotations, boxesm trails, numpy for array rel. operations

#maths for avergaes, centroid, euclidean or other distances calc (other dist may includean some-customised dist.metric lyk the..depending on the context...(human body isnt perfect polygon...))

import cv2
import math
import numpy as np
from ultralytics import YOLO    #YOLO "you only look once model"..used here for object detectn.
from tqdm import tqdm
import math



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


#video input

In [3]:
from google.colab import files
import os

uploaded = files.upload()

input_vid = "/content/" + list(uploaded.keys())[0]
annotated_vid = "/content/output_tennis_full_tracking_video.mp4"


Saving tennis_video_assignment.mp4 to tennis_video_assignment.mp4


#model configuration + parameters

In [4]:
# Configuration
player_detect_model = "yolov8n.pt"   #only 2 players hence nano model used..confidence can be adjusted for a greater accuracy or...less number of FPs (less false pos. in player detection)


player_confidence = 0.6         #confidence threshold..tried a few values here
player_max_trail = 50               # set to 0/1 to remove trails
player_max_frames_miss = 50              #how many frames a track can be missed before deactivated
player_max_detect_dist = 180        #the maximum distance to match detection to an existing track..trail





#defining segments & functions

In [5]:
#NOT all parts of the video have the 2 players actively playing tennis...
#                                      so we FILTER OUT THE SEGMENTS IN WHICH TENNIS IS BEING PLAYED


#                         MANUAL SEGMENTATION is done.
#   Criteria - NOT A REPLAY & the TENNIS COURT BEING ENTIRELY VISIBLE

In [6]:
match_segments_set_to = True
tennis_segments = [
    # (start_sec, end_sec),
    (1, 4), (10, 15), (20, 22), (24, 27), (32, 35), (41,44), (49,51),(69,73),
    (76,79), (84,86), (92,100), (107,110), (115,120), (126,129), (137,146), (158,159),
    (165,168), (173,175), (180, 183), (188, 189), (200, 206), (212,213) , (216, 218), (225, 227)
]

#the numbers are in seconds(s) unit...simple maths - Number = (x*60)+y for x:y timestamp on video

In [7]:
#for displacement/distance covered measurements lets define some vars/funcs
# bounding_box_center - avg of extreme most x,y coords of box.
# coords represents the tuple containing the coords of extreme most corners of the bounding_box

def bounding_box_center(coords):
    x1, y1, x2, y2 = coords
    return ((x1 + x2) / 2.0, (y1 + y2) / 2.0)

#EUCLIDEAN dist is considered..we MAY consider other distances as welll, depending on context..
#euclidean dist best for direct/brute distance measurement

def euclidean(a, b):
    return math.hypot(a[0] - b[0], a[1] - b[1])

#colour intialisation for the IDs/names of tennis playrs, Trails & box.

def color_for_id(i) :
    rng = np.random.RandomState(i)
    return tuple(int(x) for x in rng.randint(75, 225, size=3))
#75,225 chosen randomly for colours...


# func - in_match_segments will process the "segments" (of tennis_match) that we earlier segmented manually

def in_match_segments(frame_idx, fps):
    if not match_segments_set_to or not tennis_segments:
        return True
    t = frame_idx / max(fps, 1.0)
    for s, e in tennis_segments :
        if s <= t <= e:
            return True
    return False





#defining the classes

In [8]:
### class constructor to just note smth...
# IDs, coords (centroids), last_frame & skipped_frame (for handling abrupt interuption), path (trail of centroids), cum_distance_pixels (total distnce in pixels)
class Track:
    def __init__(self, track_id, coords, frame_idx):
        self.id = track_id
        self.coords = coords  # (x1,y1,x2,y2)
        self.centroid = bounding_box_center(coords)
        self.last_frame = frame_idx  #older values - forget the player/deactivate the trail
        self.skipped_frames = 0                  # for how many frames the player is missing (depnds on player_max_trail as well)
        self.path = [self.centroid]
        self.cum_distance_pixels = 0.0
        self.active = True

#updates the new centroid, cummulative_pixel_distance...
    def update(self, coords, frame_idx):
        new_cent = bounding_box_center(coords)
        self.cum_distance_pixels += euclidean(self.centroid, new_cent)
        self.coords = coords
        self.centroid = new_cent
        self.last_frame = frame_idx
        self.skipped_frames = 0
        self.path.append(new_cent)
        if len(self.path) > player_max_trail:
            self.path.pop(0)


#if theres an omission "within" the segmented parts..then a to handle it
#no new_ID is issued to detect_object..player in this case instantly after a disappearance n coming back into frame


#this would happen rarely..included thos because of "mistakes" in manual segmentation..

    def mark_missed(self):
        self.skipped_frames += 1
        if self.skipped_frames > player_max_frames_miss:
            self.active = False

In [9]:


class SimpleTracker:
    def __init__(self, max_distance=200):                  #max_distance - sort of opp to skipped frames.."when" to forget a player...
        self.tracks = []
        self.next_id = 1
        self.max_distance = max_distance


        #            GREEDY chosen because easiest and ASSUMPTION that Player Movement isnt THAT COMPLEX.
        #              other options are doing a IoU of the bounding_boxes, hungarian (smallest dist. finder)

    def _greedy_assign(self, active_tracks, detections):


        if not active_tracks or not detections:
            return []


        pairs = []
        for i, t in enumerate(active_tracks):
            tc = t.centroid
            for j, b in enumerate(detections):
                d = euclidean(tc, bounding_box_center(b))
                pairs.append((d, i, j))


        pairs.sort(key=lambda x: x[0])
        used_tracks, used_dets, matches = set(), set(), []
        for d, i, j in pairs:
            if d <= self.max_distance and i not in used_tracks and j not in used_dets:
                matches.append((i, j))
                used_tracks.add(i)
                used_dets.add(j)
        return matches

#first time initialisation....assigns unique IDs..updates (or deactivates if player missing)
    def step(self, detections, frame_idx):

        if not self.tracks:
            for b in detections:
                t = Track(self.next_id, b, frame_idx)
                self.next_id += 1
                self.tracks.append(t)
            return [(t.id, t.coords) for t in self.tracks]


        active_tracks = [t for t in self.tracks if t.active]


        matches = self._greedy_assign(active_tracks, detections)

        assigned_track_indices = set()
        assigned_det_indices = set()


        for i, j in matches:
            active_tracks[i].update(detections[j], frame_idx)
            assigned_track_indices.add(i)
            assigned_det_indices.add(j)


        for j, b in enumerate(detections):
            if j not in assigned_det_indices:
                t = Track(self.next_id, b, frame_idx)
                self.next_id += 1
                self.tracks.append(t)


        for idx, t in enumerate(active_tracks):
            if idx not in assigned_track_indices and t.last_frame != frame_idx:
                t.mark_missed()


        return [(t.id, t.coords) for t in self.tracks if t.last_frame == frame_idx and t.active]


#YOLO model deploy + applying segmented timeframe logic + Player Naming

In [10]:
#main model loop this...

#YOLO model loaded.
model = YOLO(player_detect_model)

cap = cv2.VideoCapture(input_vid) #input_vid loaded onto the model


#get video properties...30.0 is for fallback case.
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(annotated_vid, fourcc, fps, (width, height))

tracker = SimpleTracker(max_distance=player_max_detect_dist)
player_names = {}
# New: Persistent logical distances
logical_distances = {"A": 0.0, "B": 0.0}
trackid_to_logical = {}
frame_idx = 0

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
#            Checking if the current frame's timestamp lie in the match segments list we made earlirer..only that'll get processed

    if in_match_segments(frame_idx, fps):
        results = model(frame, verbose=False, classes=[0], conf=player_confidence)     # classes=[0]  refers to class ID 0...a 'person' class in COCO dataset.
        dets = []          # Convering detection results into a list of bounding boxes...with x1, y1, x2, y2 as coords

        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            dets.append((x1, y1, x2, y2))



        # If more than 2 detections are found..we keep only the top 2 (TOP 2 BY AREA ) (area calc by coords that we have)
#again we can cite here abt our assumption OR Problem Statement - ONLY 2 PLAYERS we wanna track.
        if len(dets) > 2:
            dets = sorted(dets, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)[:2]



    # detecttions passed to tracker so that IDs can be maintained
        updated = tracker.step(dets, frame_idx)

        for track_id, _ in updated:
          if track_id not in player_names:
            if len(player_names) == 0:
               player_names[track_id] = "Player_1"
            elif len(player_names) == 1:
               player_names[track_id] = "Player_2"

# AS A QUICK FIX ( to avoid using CNNs for accurate player detection..0
#-------------------------------------we will assign - "A" to the PLAYER ON LEFT OF THE SCREEN, "B" to the PLAYER ON RIGHT OF THE SCREEN-----------------------------------------------------------



        active_tracks = [t for t in tracker.tracks if t.active]
        if len(active_tracks) == 2:
            active_sorted = sorted(active_tracks, key=lambda t: t.centroid[0])
            left, right = active_sorted

            trackid_to_logical[left.id] = "A"
            trackid_to_logical[right.id] = "B"

            for player, track in zip(["A", "B"], [left, right]):
                logical_distances[player] += euclidean(track.path[-2], track.path[-1]) if len(track.path) > 1 else 0

        for track_id, coords in updated:
            tcolor = color_for_id(track_id)
            x1, y1, x2, y2 = map(int, coords)
            logical_id = trackid_to_logical.get(track_id, None)
            if logical_id:
                name = f"Player {logical_id}"
                dist = logical_distances[logical_id]
            else:
              #fallback mechanism if the player IDs ("A" OR "B" does NOT get assigned)
                name = f"ID {track_id}"
                dist = 0

            cv2.rectangle(frame, (x1, y1), (x2, y2), tcolor, 3)
            cv2.putText(frame, f"{name} {dist:.1f}px", (x1, max(20, y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, tcolor, 2)

            tr = next((t for t in tracker.tracks if t.id == track_id), None)

            #Trail path drawing
            if tr and len(tr.path) > 1 and player_max_trail > 1:
                for k in range(1, len(tr.path)):
                    p1 = tuple(map(int, tr.path[k-1]))
                    p2 = tuple(map(int, tr.path[k]))
                    cv2.line(frame, p1, p2, tcolor, 2)
            if tr:
                c = tuple(map(int, tr.centroid))
                cv2.circle(frame, c, 3, tcolor, -1)

    out.write(frame)
    frame_idx += 1

In [12]:
cap.release()
out.release()

#standard step to close the video files, file handling sop...


In [13]:
# final cummulative distances

final_A = [(tid, t.cum_distance_pixels) for tid, name in player_names.items() if name == "Player A"
           for t in tracker.tracks if t.id == tid]
final_B = [(tid, t.cum_distance_pixels) for tid, name in player_names.items() if name == "Player B"
           for t in tracker.tracks if t.id == tid]

#downloading final output video

In [14]:
from google.colab import files
files.download("output_tennis_full_tracking_video.mp4")

#downloading the output video file.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>